# Coursera - Capstone Project for IBM Data Science Certificate
### The battle of the neighborhoods" by Carlos Barros

### Introduction
The present laboratory is part of the asgination of week 3 of the final project of the professional Big Data certification carried out by IBM through Coursera. In this work explore, segment, and cluster the neighborhoods in the city of Barranquilla.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Import Libraries </a>

2. <a href="#item2">Sectors of the city of Barranquilla - Data and mapping </a>

3. <a href="#item3"> Use the Foursquare API to explore the neighborhoods</a>

4. <a href="#item4">Analyze each neighborhood </a>

5. <a href="#item4">Clusters neighborhood </a>

6. <a href="#item4">Examine clusters </a>

</font>
</div>




## 1. Import Libraries

In [1]:

import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

!pip install folium
# map rendering library
import folium 

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates      anaconda::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi               anaconda::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl                anaconda::openssl-1.1.1-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Libraries imported.


## 2. Sectors of the city of Barranquilla - Data and mapping.

Cluster neighborhood data was produced with Foursquare during the laboratory work of the course. A csv file was produced containing the neighborhoods around the 17 main and commercial districts of the city of Barranquilla. Now, the csv file has just been read for convenience and consolidation of the report.

In [2]:
import os
cwd = os.getcwd()
cwd

'/resources/labs/IBM DATA SCIENCE CERTIFICATE/#9 Applied Data Science Capstone/Translate spanish/WEEK 4 THE BATTLE OF NEIGHBORHOODS '

In [3]:
# Read csv file with clustered neighborhoods with geodata
bq_data  = pd.read_csv('Barrios_coord.csv') 
bq_data.head(5)

,Neighborhood,Latitude,Longitude
0,Altos De Riomar,11.015578,-74.820644
1,Miramar,11.003472,-74.835111
2,Andalucia,11.016125,-74.815162
3,Altos Del Limon,11.013992,-74.826282
4,El Golf,11.008695,-74.808828


In [4]:
# get the coordinates of Barranquilla
address = 'Barranquilla,Atlántico,Colombia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 10.9799669, -74.8013085.


In [5]:
latitude= 11.008799
longitude= -74.805004 

# create map of Barranquilla using latitude and longitude values
map_bq = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood in zip(bq_data['Latitude'], bq_data['Longitude'], bq_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bq)  
    
map_bq

In [6]:
# save the map as HTML file
map_bq.save('map_bq.html')

## 3. Use the Foursquare API to explore the neighborhoods

In [7]:
# credentails
CLIENT_ID = 'N1FXT1RQOUZAZCMKURH3PFJYM5VGHNSMZQHK4JXWVQ5DDI1Y' # your Foursquare ID
CLIENT_SECRET = 'PLQF0OEC3F2QSJREOWM2YSTJTOAWWAAP324E1EWBFA0AGHTG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 5000
LIMIT = 200

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N1FXT1RQOUZAZCMKURH3PFJYM5VGHNSMZQHK4JXWVQ5DDI1Y
CLIENT_SECRET:PLQF0OEC3F2QSJREOWM2YSTJTOAWWAAP324E1EWBFA0AGHTG


##### **Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [8]:
radius = 5000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(bq_data['Latitude'], bq_data['Longitude'], bq_data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [9]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1523, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Altos De Riomar,11.015578,-74.820644,Margarita Saieh De Jassir,11.013333,-74.819209,Dessert Shop
1,Altos De Riomar,11.015578,-74.820644,Parque Bulevar Buenavista,11.015699,-74.826604,Park
2,Altos De Riomar,11.015578,-74.820644,Bernny's Pizza,11.011099,-74.822762,Pizza Place
3,Altos De Riomar,11.015578,-74.820644,Centro Comercial VIVA Barranquilla,11.008106,-74.821036,Shopping Mall
4,Altos De Riomar,11.015578,-74.820644,Centro Comercial Buenavista I,11.013285,-74.827622,Shopping Mall


##### **Let's check how many venues were returned for each neighorhood**

In [10]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alto Prado,100,100,100,100,100,100
Altos De Riomar,100,100,100,100,100,100
Altos Del Limon,100,100,100,100,100,100
Andalucia,100,100,100,100,100,100
Boston,100,100,100,100,100,100
Ciudad Jardin,100,100,100,100,100,100
El Golf,100,100,100,100,100,100
El Recreo,100,100,100,100,100,100
El Tabor,52,52,52,52,52,52


##### **Let's find out how many unique categories can be curated from all the returned venues**

In [11]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 73 uniques categories.


In [12]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Dessert Shop', 'Park', 'Pizza Place', 'Shopping Mall',
       'Italian Restaurant', 'Gastropub', 'Hotel', 'Department Store',
       'French Restaurant', 'Coffee Shop', 'Bakery', 'Ice Cream Shop',
       'Falafel Restaurant', 'Bookstore', 'Middle Eastern Restaurant',
       'Brazilian Restaurant', 'Burger Joint', 'Mexican Restaurant',
       'Martial Arts Dojo', 'Pub', 'Japanese Restaurant',
       'Sushi Restaurant', 'Argentinian Restaurant', 'Supermarket',
       'Multiplex', 'Clothing Store', 'Bar', 'Bed & Breakfast',
       'Caribbean Restaurant', 'Golf Course', 'Boutique',
       'Mediterranean Restaurant', 'Peruvian Restaurant', 'Restaurant',
       'Shoe Store', 'Sandwich Place', 'Theme Park Ride / Attraction',
       'Fast Food Restaurant', 'Waterfront', 'Fried Chicken Joint',
       'Roof Deck', 'Gym / Fitness Center', 'Cocktail Bar', 'Zoo',
       'Arepa Restaurant', 'Monument / Landmark', 'Juice Bar',
       'Sporting Goods Shop', 'Salad Place', 'Snack Place'], dtype

In [13]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

## 4. Analyze Each Neighborhood

In [14]:
# one hot encoding
bq_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bq_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bq_onehot.columns[-1]] + list(bq_onehot.columns[:-1])
bq_onehot = bq_onehot[fixed_columns]

print(bq_onehot.shape)
bq_onehot.head()

(1523, 74)


,Neighborhoods,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Stadium,Bed & Breakfast,Bookstore,Boutique,Brazilian Restaurant,Burger Joint,Café,Caribbean Restaurant,Casino,Clothing Store,Cocktail Bar,Coffee Shop,College Soccer Field,Department Store,Dessert Shop,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Fried Chicken Joint,Gastropub,Golf Course,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Multiplex,Museum,Nightclub,Park,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pool,Pub,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Theater,Theme Park Ride / Attraction,Toll Plaza,Waterfront,Zoo
0,Altos De Riomar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Altos De Riomar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Altos De Riomar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Altos De Riomar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Altos De Riomar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
bq_grouped = bq_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bq_grouped.shape)
bq_grouped.head(5)

(17, 74)


,Neighborhoods,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Stadium,Bed & Breakfast,Bookstore,Boutique,Brazilian Restaurant,Burger Joint,Café,Caribbean Restaurant,Casino,Clothing Store,Cocktail Bar,Coffee Shop,College Soccer Field,Department Store,Dessert Shop,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Fried Chicken Joint,Gastropub,Golf Course,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Multiplex,Museum,Nightclub,Park,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pool,Pub,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Theater,Theme Park Ride / Attraction,Toll Plaza,Waterfront,Zoo
0,Alto Prado,0.0,0.01,0.01,0.0,0.00,0.07,0.01,0.00,0.00,0.01,0.00,0.01,0.04,0.00,0.02,0.01,0.00,0.00,0.03,0.01,0.03,0.02,0.03,0.02,0.02,0.00,0.01,0.01,0.01,0.00,0.05,0.05,0.05,0.01,0.01,0.00,0.01,0.01,0.02,0.04,0.00,0.01,0.00,0.00,0.03,0.01,0.0,0.10,0.00,0.0,0.02,0.04,0.01,0.0,0.00,0.00,0.01,0.00,0.03,0.0,0.0,0.00,0.01,0.0,0.01,0.02,0.03,0.01,0.01,0.00,0.0,0.01,0.01
1,Altos De Riomar,0.0,0.01,0.01,0.0,0.00,0.06,0.01,0.00,0.01,0.01,0.01,0.01,0.05,0.00,0.01,0.00,0.02,0.01,0.03,0.00,0.04,0.02,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.06,0.06,0.06,0.01,0.01,0.00,0.01,0.01,0.01,0.03,0.01,0.01,0.00,0.00,0.03,0.01,0.0,0.10,0.00,0.0,0.01,0.02,0.01,0.0,0.01,0.00,0.00,0.01,0.03,0.0,0.0,0.00,0.00,0.0,0.00,0.02,0.04,0.00,0.00,0.01,0.0,0.01,0.01
2,Altos Del Limon,0.0,0.01,0.01,0.0,0.00,0.06,0.01,0.00,0.01,0.01,0.01,0.01,0.05,0.00,0.01,0.00,0.02,0.01,0.03,0.00,0.04,0.02,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.06,0.06,0.06,0.01,0.01,0.00,0.01,0.01,0.01,0.04,0.01,0.01,0.00,0.00,0.03,0.01,0.0,0.09,0.00,0.0,0.01,0.02,0.01,0.0,0.01,0.00,0.00,0.01,0.03,0.0,0.0,0.01,0.00,0.0,0.00,0.02,0.04,0.00,0.00,0.01,0.0,0.01,0.00
3,Andalucia,0.0,0.01,0.01,0.0,0.00,0.06,0.01,0.00,0.01,0.01,0.01,0.01,0.05,0.00,0.01,0.00,0.02,0.00,0.03,0.00,0.04,0.02,0.03,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.06,0.06,0.05,0.01,0.01,0.00,0.01,0.01,0.02,0.04,0.01,0.01,0.00,0.00,0.02,0.01,0.0,0.09,0.00,0.0,0.01,0.02,0.01,0.0,0.01,0.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,0.0,0.01,0.02,0.04,0.01,0.00,0.01,0.0,0.01,0.01
4,Boston,0.0,0.01,0.01,0.0,0.01,0.06,0.01,0.01,0.00,0.00,0.00,0.01,0.03,0.01,0.03,0.01,0.00,0.00,0.03,0.01,0.01,0.01,0.02,0.03,0.01,0.00,0.01,0.01,0.01,0.01,0.02,0.03,0.05,0.01,0.01,0.02,0.01,0.01,0.02,0.04,0.00,0.00,0.01,0.01,0.03,0.01,0.0,0.09,0.03,0.0,0.02,0.05,0.01,0.0,0.00,0.01,0.02,0.00,0.03,0.0,0.0,0.00,0.01,0.0,0.02,0.01,0.02,0.01,0.01,0.00,0.0,0.01,0.01


In [16]:
len(bq_grouped[bq_grouped["Peruvian Restaurant"] > 0])


13

##### **Create a new DataFrame for Peruvian Restaurant data only**

In [17]:
bq_peruvian = bq_grouped[["Neighborhoods","Peruvian Restaurant"]]

In [18]:
bq_peruvian.head()

,Neighborhoods,Peruvian Restaurant
0,Alto Prado,0.01
1,Altos De Riomar,0.01
2,Altos Del Limon,0.01
3,Andalucia,0.01
4,Boston,0.01


##  5. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Barranquilla into 3 clusters.


In [19]:
# set number of clusters
kclusters = 4

bq_clustering = bq_peruvian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bq_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 0, 3], dtype=int32)

In [20]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bq_merged = bq_peruvian.copy()

# add clustering labels
bq_merged["Cluster Labels"] = kmeans.labels_

In [21]:
bq_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bq_merged.head()

,Neighborhood,Peruvian Restaurant,Cluster Labels
0,Alto Prado,0.01,3
1,Altos De Riomar,0.01,3
2,Altos Del Limon,0.01,3
3,Andalucia,0.01,3
4,Boston,0.01,3


In [22]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bq_merged = bq_merged.join(bq_data.set_index("Neighborhood"), on="Neighborhood")

print(bq_merged.shape)
bq_merged.head() # check the last columns!

(17, 5)


,Neighborhood,Peruvian Restaurant,Cluster Labels,Latitude,Longitude
0,Alto Prado,0.01,3,11.001832,-74.809796
1,Altos De Riomar,0.01,3,11.015578,-74.820644
2,Altos Del Limon,0.01,3,11.013992,-74.826282
3,Andalucia,0.01,3,11.016125,-74.815162
4,Boston,0.01,3,10.986946,-74.793942


In [23]:
# sort the results by Cluster Labels
print(bq_merged.shape)
bq_merged.sort_values(["Cluster Labels"], inplace=True)
bq_merged

(17, 5)


,Neighborhood,Peruvian Restaurant,Cluster Labels,Latitude,Longitude
8,El Tabor,0.00,0,11.002278,-74.828940
14,Villa Campestre,0.00,0,11.023403,-74.862230
13,Riomar,0.00,0,11.011865,-74.831650
16,Villa Del Mar,0.00,0,11.005024,-74.827649
4,Boston,0.01,3,10.986946,-74.793942
5,Ciudad Jardin,0.01,3,10.994720,-74.818642
6,El Golf,0.01,3,11.008695,-74.808828
7,El Recreo,0.01,3,10.980112,-74.800932
15,Villa Country,0.01,3,11.006075,-74.804908
9,Las Tres Aves Marias,0.01,3,11.021004,-74.808954


##### **Finally, let's visualize the resulting clusters**

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bq_merged['Latitude'], bq_merged['Longitude'], bq_merged['Neighborhood'], bq_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
# save the map as HTML file
map_clusters.save('map_clusters.jpg')

## 6. Examine Clusters

#### Cluster 0

In [26]:
bq_merged.loc[bq_merged['Cluster Labels'] == 0]

,Neighborhood,Peruvian Restaurant,Cluster Labels,Latitude,Longitude
8,El Tabor,0.0,0,11.002278,-74.828940
14,Villa Campestre,0.0,0,11.023403,-74.862230
13,Riomar,0.0,0,11.011865,-74.831650
16,Villa Del Mar,0.0,0,11.005024,-74.827649


#### Cluster 1

In [27]:
bq_merged.loc[bq_merged['Cluster Labels'] == 1]

,Neighborhood,Peruvian Restaurant,Cluster Labels,Latitude,Longitude
